# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

In [1]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'biaflows.neubias.org',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '11244427',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Please enter the public key: ········
Please enter the private key: ········


Import CytomineJob and Job and update the status information. Set the problem class to particle-tracking (PrtTrk).

In [3]:
import sys
from subprocess import call

from cytomine.models import Job
from neubiaswg5.helpers import NeubiasJob, prepare_data, upload_data, upload_metrics
from neubiaswg5 import CLASS_PRTTRK

jobID=-666
with NeubiasJob.from_cli(argv) as nj:
    nj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
    jobID = nj.job.id
    
problem_cls = CLASS_PRTTRK

[2019-09-23 12:20:25,031][INFO] [GET] [currentuser] CURRENT USER - 3002336 : guest | 200 OK
[2019-09-23 12:20:25,111][INFO] [GET] [project] 1695226 : NUCLEI-TRACKING-NODIVISION | 200 OK
[2019-09-23 12:20:25,189][INFO] [GET] [software] 11244427 : NucleiTracking-ImageJ | 200 OK
[2019-09-23 12:20:25,255][INFO] [GET] [softwareparameter collection] 8 objects | 200 OK
[2019-09-23 12:20:25,493][INFO] [POST] [job] 21010021 : None | 200 OK
[2019-09-23 12:20:25,569][INFO] [GET] [user] 21010027 : JOB[guest ], 2019-09-23 12:20:25:452 | 200 OK
[2019-09-23 12:20:25,638][INFO] [GET] [currentuser] CURRENT USER - 21010027 : JOB[guest ], 2019-09-23 12:20:25:452 | 200 OK
[2019-09-23 12:20:25,640][INFO] Job (id:21010021) status update: "None" (status: RUNNING, progress: 0%)
[2019-09-23 12:20:25,756][INFO] [PUT] [job] 21010021 : None | 200 OK
[2019-09-23 12:20:25,847][INFO] [POST] [jobparameter] 21010034 : ij_gauss_radius | 200 OK
[2019-09-23 12:20:25,934][INFO] [POST] [jobparameter] 21010040 : ij_threshol

#### Create local directories and download images

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [4]:
in_images, gt_images, in_path, gt_path, out_path, tmp_path = prepare_data(problem_cls, nj, is_2d=False, **nj.flags)

[2019-09-23 12:22:04,483][INFO] Job (id:21010021) status update: "Downloading images (to /home/jovyan/21010021/in)..." (status: SUCCESS/TERMINATED, progress: 0%)
[2019-09-23 12:22:04,677][INFO] [PUT] [job] 21010021 : None | 200 OK
[2019-09-23 12:22:04,748][INFO] [GET] [imagegroup collection] 2 objects | 200 OK
[2019-09-23 12:22:06,732][INFO] File downloaded successfully from http://biaflows.neubias.org/api/imagegroup/4042304/download
[2019-09-23 12:22:08,581][INFO] File downloaded successfully from http://biaflows.neubias.org/api/imagegroup/4042708/download


Check the downloaded input and ground-truth images. In this case there is only one time-series as input image and one time-series as ground-truth image.

In [23]:
print(in_path)
!ls -alh $in_path
print(gt_path)
!ls -alh $gt_path

/home/jovyan/21010021/in
total 13M
drwxr-xr-x. 2 jovyan root 4.0K Sep 23 12:22 .
drwxr-xr-x. 6 jovyan root 4.0K Sep 23 12:22 ..
-rw-r--r--. 1 jovyan root  13M Sep 23 12:22 4042304.tif
/home/jovyan/21010021/ground_truth
total 25M
drwxr-xr-x. 2 jovyan root 4.0K Sep 23 12:22 .
drwxr-xr-x. 6 jovyan root 4.0K Sep 23 12:22 ..
-rw-r--r--. 1 jovyan root  25M Sep 23 12:22 4042304.tif


#### Call the image analysis workflow 

In [24]:
nj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro NucleiTracking.ijm " \
		"\"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, nj.parameters.ij_gauss_radius, nj.parameters.ij_threshold, nj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./Fiji.app")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    nj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
nj.job.update(progress=30, statusComment="Workflow finished...")

[2019-09-23 12:35:28,374][INFO] Job (id:21010021) status update: "Launching workflow..." (status: SUCCESS/TERMINATED, progress: 1%)
[2019-09-23 12:35:28,791][INFO] [PUT] [job] 21010021 : None | 200 OK


FileNotFoundError: [Errno 2] No such file or directory: './Fiji.app': './Fiji.app'